# 서울시 교통사고 시각화

In [27]:
import pandas as pd

In [28]:
traffic_Seoul = pd.read_csv("../Data/newSeoul_2005_2019.csv")
traffic_Seoul.head()

,년도,월,자치구명,발생건수,사망자수,부상자수
0,2005,1,종로구,93,2,138
1,2005,2,종로구,84,3,125
2,2005,3,종로구,117,0,142
3,2005,4,종로구,138,2,212
4,2005,5,종로구,145,2,207


In [29]:
# 2019년 데이터만 추출하기
traffic_Seoul_2019 = traffic_Seoul[traffic_Seoul['년도'] == 2019]
traffic_Seoul_2019

,년도,월,자치구명,발생건수,사망자수,부상자수
4200,2019,1,종로구,87,1,125
4201,2019,2,종로구,66,1,84
4202,2019,3,종로구,87,2,122
4203,2019,4,종로구,85,0,131
4204,2019,5,종로구,112,1,158
...,...,...,...,...,...,...
4495,2019,8,강동구,127,2,175
4496,2019,9,강동구,98,2,137
4497,2019,10,강동구,108,1,144
4498,2019,11,강동구,146,0,199


In [30]:
# index 정리하기
traffic_Seoul_2019.reset_index(drop=True, inplace=True,)
traffic_Seoul_2019

,년도,월,자치구명,발생건수,사망자수,부상자수
0,2019,1,종로구,87,1,125
1,2019,2,종로구,66,1,84
2,2019,3,종로구,87,2,122
3,2019,4,종로구,85,0,131
4,2019,5,종로구,112,1,158
...,...,...,...,...,...,...
295,2019,8,강동구,127,2,175
296,2019,9,강동구,98,2,137
297,2019,10,강동구,108,1,144
298,2019,11,강동구,146,0,199


In [31]:
import numpy as np

In [32]:
# 자치구별 발생건수, 사망자수, 부상자수 집계하기
# traffic_analysis = pd.pivot_table(traffic_Seoul_2019, index="자치구명", aggfunc=np.sum)
traffic_analysis = pd.pivot_table(
    traffic_Seoul_2019[['자치구명','발생건수', '부상자수', '사망자수']], index="자치구명", aggfunc=np.sum,)
traffic_analysis.head()

,발생건수,부상자수,사망자수
자치구명,,,
강남구,3722,5182,14
강동구,1414,1910,11
강북구,1277,1706,7
강서구,1829,2491,20
관악구,1363,1755,10


In [33]:
# traffic_Seoul_2019[['발생건수', '부상자수', '사망자수']]

In [45]:
# 위도, 경도 데이터 불러오기
seoul_limit = pd.read_csv("../Data/seoul.csv", encoding='euc=kr')
seoul_limit.columns = ['자치구명', 'lng', 'lat']
seoul_limit.head()

,자치구명,lng,lat
0,강남구,127.0475,37.51731
1,강동구,127.1238,37.53013
2,강북구,127.0255,37.63975
3,관악구,126.9515,37.47834
4,구로구,126.8875,37.49547


In [46]:
# traffic_Seoul_2019(traffic_analysis)와 seoul_limit 병합하기
data_result = pd.merge(traffic_analysis, seoul_limit, on='자치구명')
data_result.head()

,자치구명,발생건수,부상자수,사망자수,lng,lat
0,강남구,3722,5182,14,127.0475,37.51731
1,강동구,1414,1910,11,127.1238,37.53013
2,강북구,1277,1706,7,127.0255,37.63975
3,강서구,1829,2491,20,126.8496,37.55094
4,관악구,1363,1755,10,126.9515,37.47834


In [47]:
data_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자치구명    25 non-null     object 
 1   발생건수    25 non-null     int64  
 2   부상자수    25 non-null     int64  
 3   사망자수    25 non-null     int64  
 4   lng     25 non-null     float64
 5   lat     25 non-null     float64
dtypes: float64(2), int64(3), object(1)
memory usage: 1.4+ KB


In [48]:
# folium을 사용하기 위해 int를 float으로 변환해야 한다. 
data_result[['발생건수', '부상자수', '사망자수']] = data_result[['발생건수', '부상자수', '사망자수']].astype('float64')
data_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 24
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자치구명    25 non-null     object 
 1   발생건수    25 non-null     float64
 2   부상자수    25 non-null     float64
 3   사망자수    25 non-null     float64
 4   lng     25 non-null     float64
 5   lat     25 non-null     float64
dtypes: float64(5), object(1)
memory usage: 1.4+ KB


### 자치구의 구청을 표시하기

In [49]:
import folium

In [54]:
# 1. 화면에 지도를 먼저 그린다 .
map = folium.Map(
    location=[37.5502, 126.982],
    zoom_start=11
)

# 2. 마커를 찍는다 . 
# 위도와 경도 순서써준거랑 똑같이 folium Marker에도 똑같이 순서대로 써줘야함. 안써줘서 오류.
# for name, lat, lng in zip(data_result.발생건수, data_result.lat, data_result.lng):
#     folium.Marker([lat,lng], popup=name).add_to(map)

# 선생님의 방법으로하면 인덱스로 for문을 돌리고 팝업에 자치구명을 넣어주었다.  
for n in data_result.index:
    folium.Marker([data_result['lat'][n], data_result['lng'][n]],
                popup=data_result['자치구명'][n]
    ).add_to(map)



# 3. 출력
map

### 발생건수

In [89]:
# 자치구별 교통사고 발생건수를 표시하기
# 1. 화면에 지도를 먼저 그린다 .
map = folium.Map(
    location=[37.5502, 126.982],
    zoom_start=11
)

# 2. 마커를 찍는다 .
# 위도와 경도 순서써준거랑 똑같이 folium Marker에도 똑같이 순서대로 써줘야함. 안써줘서 오류.
# for name, lat, lng in zip(data_result.발생건수, data_result.lat, data_result.lng):
#     folium.Marker([lat,lng], popup=name).add_to(map)

# 선생님의 방법으로하면 인덱스로 for문을 돌리고 팝업에 자치구명을 넣어주었다.
for n in data_result.index:
    folium.Marker([data_result['lat'][n], data_result['lng'][n]],
                  popup=data_result['자치구명'][n]
                  ).add_to(map)

for name, lat, lng, rad in zip(data_result.자치구명, data_result.lat, data_result.lng, data_result.발생건수,):
    folium.CircleMarker(
                    [lat,lng],
                     radius = rad / 100,
                     color = 'red',
                    #  fill = True,
                    fill_color='red',
                     fill_opacity = (rad /data_result.발생건수.max()) * 0.7,
                     popup=name).add_to(map)


# 3. 출력
map


### 사망건수

In [110]:
map = folium.Map(
    location=[37.5502, 126.982],
    zoom_start=11
)
for name, lat, lng, dead in zip(data_result.자치구명, data_result.lat, data_result.lng, data_result.사망자수):

    folium.Marker([lat,lng], popup=name).add_to(map)

folium.CircleMarker(
                        [data_result.loc[data_result.사망자수 == data_result.사망자수.max(), "lat"],
                         data_result.loc[data_result.사망자수 == data_result.사망자수.max(), "lng"]],
                        radius = data_result.사망자수.max() * 1.9,
                        color = 'purple',
                        fill_color= 'purple',
                        fill_opacity= 0.7,
                        popup=name).add_to(map)

folium.CircleMarker(
                        [data_result.loc[data_result.사망자수 == data_result.사망자수.min(), "lat"],
                         data_result.loc[data_result.사망자수 == data_result.사망자수.min(), "lng"]],
                        radius = data_result.사망자수.min() * 1.9,
                        color = 'purple',
                        fill_color= 'purple',
                        fill_opacity= 0.7,
                        popup=name).add_to(map)

map

In [111]:
data_result2 = data_result.sort_values("사망자수", ascending=False)
data_result2.reset_index(drop=True,inplace=True)
data_result2.tail()


,자치구명,발생건수,부상자수,사망자수,lng,lat
20,종로구,1133.0,1559.0,6.0,126.9793,37.57323
21,동작구,1406.0,2021.0,5.0,126.9393,37.51243
22,중구,1184.0,1615.0,5.0,127.0032,37.55273
23,도봉구,852.0,1163.0,4.0,127.0478,37.66873
24,광진구,973.0,1316.0,3.0,127.0824,37.53862


In [114]:
map = folium.Map(
    location=[37.5502, 126.982],
    zoom_start=11
)
for n in data_result2.index:

    folium.Marker(data_result2['lat'][n], data_result2['lng']
                  [n], popup=data_result2['자치구명'][n]).add_to(map)

folium.CircleMarker(
    [data_result2['lat'][0], data_result2['lng'][0]],
    radius=data_result2['사망자수'][0] * 1.9,
    fill=True,
    color='purple',
    fill_color='purple',
    fill_opacity=0.7,
    popup=data_result2['사망자수'][0]).add_to(map)

folium.CircleMarker(
    [data_result2['lat'][len(data_result2)-1],
     data_result2['lng'][len(data_result2)-1]],
    radius=data_result2['사망자수'][len(data_result2)-1] * 1.9,
    fill= True,
    color='purple',
    fill_color='purple',
    fill_opacity=0.7,
    popup=data_result2['사망자수'][len(data_result2)-1]).add_to(map)

map


TypeError: __init__() got multiple values for argument 'popup'